#Implementando Árvore de Busca Binária com Python

## 📚 Explicação das Classes `MapBase` e `TreeMap`

Este código define duas classes principais: **`MapBase`** e **`TreeMap`**.

### 🔑 Classe `MapBase`
A classe `MapBase` serve como uma classe base para implementar mapas (**maps**), que são estruturas de dados que armazenam pares de chave-valor. Ela fornece métodos para:
- **Adicionar** elementos,
- **Remover** elementos, e
- **Buscar** elementos no mapa.

### 🌳 Classe `TreeMap`
A classe `TreeMap` é uma implementação concreta de um mapa utilizando uma **árvore binária de busca** (**BST - Binary Search Tree**).  
Ela **herda** de `LinkedBinaryTree`, que é outra classe base que define a interface para a implementação de árvores binárias encadeadas.

A classe `LinkedBinaryTree` fornece métodos para:
- Criar a árvore,
- Modificar a árvore,
- Percorrer a árvore (**traversal**).

Além disso, `LinkedBinaryTree` contém:
- Uma classe interna chamada **`Position`**, que representa uma posição na árvore.
- Uma classe interna chamada **`Node`**, que representa um nó na árvore.

### 🛠️ Métodos Abstratos em `MapBase`
A classe `MapBase` contém dois métodos **não implementados**:
- `_validate`
- `_make_position`

Esses métodos são utilizados para:
- Validar posições no mapa,
- Criar novas posições a partir dos nós.

A classe `TreeMap` **herda** esses métodos de suas classes base, mas **não os implementa diretamente**.  
Em vez disso, ela **depende das implementações** fornecidas pelas classes pai.

### 🧩 Métodos Específicos da `TreeMap`
A classe `TreeMap` também contém vários métodos específicos para a estrutura de dados **BST**:
- Por exemplo, o método `subtree_search` realiza uma operação de busca na subárvore enraizada em um determinado nó.
- Há também métodos para **adicionar** e **remover** elementos da árvore.
- E métodos para **obter** e **definir** valores a partir da chave.

### 🧱 Estrutura Geral
De forma geral, este código fornece um **framework flexível e extensível** para implementar mapas usando diferentes estruturas de dados:
- A classe `MapBase` serve como uma **interface comum** para todas as implementações de mapas.
- A classe `TreeMap` é uma implementação específica que utiliza uma **BST** para armazenar elementos.

---

### 🗂️ Resumo Rápido

| Classe       | Descrição                                       | Herança/Relacionamento                   |
|--------------|--------------------------------------------------|------------------------------------------|
| `MapBase`    | Classe base para mapas, com métodos para manipular pares chave-valor | Base para `TreeMap`                      |
| `TreeMap`    | Implementação de mapa usando árvore BST          | Herda de `LinkedBinaryTree` e `MapBase`  |
| `LinkedBinaryTree` | Classe base para árvores binárias ligadas      | Contém `Node` e `Position`                |
| `Node`       | Representa um nó na árvore                       | Classe interna de `LinkedBinaryTree`      |
| `Position`   | Representa uma posição na árvore                 | Classe interna de `LinkedBinaryTree`      |
| `_validate`, `_make_position` | Métodos auxiliares não implementados em `MapBase` | Implementados nas classes herdadas        |



In [ ]:
class MapBase:
    """Classe base para mapas que armazena pares chave-valor."""

    class Item:
        __slots__ = 'key', 'value'

        def __init__(self, key, value):
            self.key = key
            self.value = value

        def __lt__(self, other):
            return self.key < other.key

        def __repr__(self):
            return f"({self.key}, {self.value})"


class LinkedBinaryTree:
    """Árvore binária de busca ligada."""

    class Node:
        __slots__ = 'element', 'parent', 'left', 'right'

        def __init__(self, element, parent=None, left=None, right=None):
            self.element = element
            self.parent = parent
            self.left = left
            self.right = right

    class Position:
        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node.element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p deve ser do tipo Position")
        if p._container is not self:
            raise ValueError("p não pertence a esta árvore")
        if p._node.parent is p._node:
            raise ValueError("p foi desativado")
        return p._node

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    def __init__(self):
        self._root = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def root(self):
        return self._make_position(self._root)

    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node.parent)

    def left(self, p):
        node = self._validate(p)
        return self._make_position(node.left)

    def right(self, p):
        node = self._validate(p)
        return self._make_position(node.right)

    def add_root(self, e):
        if self._root is not None:
            raise ValueError("Raiz já existe")
        self._size = 1
        self._root = self.Node(e)
        return self._make_position(self._root)

    def add_left(self, p, e):
        node = self._validate(p)
        if node.left is not None:
            raise ValueError("Filho esquerdo já existe")
        self._size += 1
        node.left = self.Node(e, parent=node)
        return self._make_position(node.left)

    def add_right(self, p, e):
        node = self._validate(p)
        if node.right is not None:
            raise ValueError("Filho direito já existe")
        self._size += 1
        node.right = self.Node(e, parent=node)
        return self._make_position(node.right)

    def replace(self, p, e):
        node = self._validate(p)
        old = node.element
        node.element = e
        return old

    def delete(self, p):
        node = self._validate(p)
        if node.left and node.right:
            raise ValueError("O nó tem dois filhos")
        child = node.left if node.left else node.right
        if child is not None:
            child.parent = node.parent
        if node is self._root:
            self._root = child
        else:
            parent = node.parent
            if node is parent.left:
                parent.left = child
            else:
                parent.right = child
        self._size -= 1
        node.parent = node  # marca como removido
        return node.element


class TreeMap(LinkedBinaryTree, MapBase):
    """Mapa ordenado usando uma árvore binária de busca."""

    class Position(LinkedBinaryTree.Position):
        def key(self):
            return self.element().key

        def value(self):
            return self.element().value

    def subtree_search(self, p, k):
        if k == p.key():
            return p
        elif k < p.key():
            if self.left(p) is not None:
                return self.subtree_search(self.left(p), k)
        else:
            if self.right(p) is not None:
                return self.subtree_search(self.right(p), k)
        return p

    def subtree_first_position(self, p):
        while self.left(p) is not None:
            p = self.left(p)
        return p

    def subtree_last_position(self, p):
        while self.right(p) is not None:
            p = self.right(p)
        return p

    def first(self):
        return self.subtree_first_position(self.root()) if not self.is_empty() else None

    def last(self):
        return self.subtree_last_position(self.root()) if not self.is_empty() else None

    def before(self, p):
        if self.left(p):
            return self.subtree_last_position(self.left(p))
        walk = p
        above = self.parent(walk)
        while above is not None and walk == self.left(above):
            walk = above
            above = self.parent(walk)
        return above

    def after(self, p):
        if self.right(p):
            return self.subtree_first_position(self.right(p))
        walk = p
        above = self.parent(walk)
        while above is not None and walk == self.right(above):
            walk = above
            above = self.parent(walk)
        return above

    def find_position(self, k):
        return None if self.is_empty() else self.subtree_search(self.root(), k)

    def find_min(self):
        p = self.first()
        return (p.key(), p.value()) if p else None

    def find_ge(self, k):
        p = self.find_position(k)
        if p and p.key() < k:
            p = self.after(p)
        return (p.key(), p.value()) if p else None

    def find_range(self, start, stop):
        if not self.is_empty():
            p = self.find_position(start) if start is not None else self.first()
            if p and start is not None and p.key() < start:
                p = self.after(p)
            while p is not None and (stop is None or p.key() < stop):
                yield (p.key(), p.value())
                p = self.after(p)

    def __getitem__(self, k):
        if self.is_empty():
            raise KeyError(f"Key Error: {k}")
        p = self.subtree_search(self.root(), k)
        if k != p.key():
            raise KeyError(f"Key Error: {k}")
        return p.value()

    def __setitem__(self, k, v):
        if self.is_empty():
            self.add_root(self.Item(k, v))
        else:
            p = self.subtree_search(self.root(), k)
            if k == p.key():
                p._node.element.value = v
            else:
                item = self.Item(k, v)
                if k < p.key():
                    self.add_left(p, item)
                else:
                    self.add_right(p, item)

    def __delitem__(self, k):
        if not self.is_empty():
            p = self.subtree_search(self.root(), k)
            if k == p.key():
                self.delete(p)
                return
        raise KeyError(f"Key Error: {k}")

    def delete(self, p):
        node = self._validate(p)
        if self.left(p) and self.right(p):
            replacement = self.subtree_last_position(self.left(p))
            self.replace(p, replacement.element())
            p = replacement
        parent = self.parent(p)
        LinkedBinaryTree.delete(self, p)
        return parent

    def __iter__(self):
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)


####Testando o código e usando a biblioteca Pandas

In [ ]:
import pandas as pd

# Inicializando o mapa
tree = TreeMap()

# Inserindo elementos
tree[50] = 'maçã'
tree[30] = 'banana'
tree[70] = 'laranja'
tree[20] = 'kiwi'
tree[40] = 'uva'
tree[60] = 'pera'
tree[80] = 'melão'

# Acessando valores por chave
ex1 = tree[30]
ex2 = tree[80]

# Atualizando um valor
tree[30] = 'banana-prata'
ex3 = tree[30]

# Iterando sobre as chaves em ordem crescente
ordem_crescente = list(tree)

# Encontrando mínimo
minimo = tree.find_min()

# Encontrando o menor >= chave
ge_45 = tree.find_ge(45)

# Iterando sobre um intervalo
intervalo = list(tree.find_range(30, 70))

# Removendo um elemento
del tree[60]
apos_remocao = list(tree)

# Exibindo os resultados
df = pd.DataFrame({
    "Operação": [
        "tree[30]",
        "tree[80]",
        "tree[30] = 'banana-prata'",
        "list(tree)",
        "tree.find_min()",
        "tree.find_ge(45)",
        "list(tree.find_range(30, 70))",
        "del tree[60]; list(tree)"
    ],
    "Resultado": [
        ex1,
        ex2,
        ex3,
        ordem_crescente,
        minimo,
        ge_45,
        intervalo,
        apos_remocao
    ]
})

df


#Implementação em Java

# 🌳 Principais Funcionalidades da Árvore

- **Classe `Entry`**  
  Representa um par de **chave–valor**, com métodos para:  
  - `getKey()`: obter a chave  
  - `getValue()`: obter o valor  

- **Classe `Node`**  
  Representa um **nó da árvore**, contendo:  
  - Uma **entrada** (`Entry<K, V>`)  
  - Referências para os **filhos esquerdo e direito**  
  - Referência para o **pai**  

- **Método `get(K key)`**  
  Busca a **chave** na árvore e retorna o **valor associado**.

- **Método `put(K key, V value)`**  
  Insere um novo **par chave–valor** ou **atualiza** o valor se a chave já existir.

- **Método `remove(K key)`**  
  Remove uma chave da árvore, tratando diferentes casos:  
  - Quando o nó é **folha**  
  - Quando o nó possui **um único filho**  
  - Quando o nó possui **dois filhos**  

---

## 🔎 Métodos Auxiliares

- **`subtreeSearch(Node root, K key)`**  
  Encontra um nó com uma determinada chave em uma **subárvore**.

- **`subtreeMin(Node root)`**  
  Retorna o nó com a **menor chave** em uma subárvore.




```java
public class TreeMap<K extends Comparable<K>, V> {
    // Classe interna Entry que armazena a chave (K) e o valor (V)
    public class Entry {
    private K key;
    private V value;

    public Entry(K key, V value) {
        this.key = key;
        this.value = value;
    }

    public K getKey() {
        return key;
    }

    public V getValue() {
        return value;
    }

    @Override
    public String toString() {
        return key + " = " + value;
    }
}

    // Classe interna Node que representa um nó da árvore, contendo uma Entry e ponteiros para filho esquerdo,
    // filho direito e pai.
    private class Node {
        Entry entry;
        Node left, right, parent;
        Node(Entry entry, Node parent) {
            this.entry = entry;
            this.parent = parent;
        }
    }
    // Variáveis de instância: raiz da árvore (inicialmente nula), tamanho da árvore.
    private Node root;
    private int size = 0;
    // Método para buscar uma chave na árvore
    public V get(K key) {
        Node node = subtreeSearch(root, key);
        return (node != null && node.entry.key.equals(key)) ? node.entry.value : null;
    }
    // Método para inserir um novo par de chave-valor na árvore
    public void put(K key, V value) {
        if (root == null) {  // Caso em que a árvore está vazia
            root = new Node(new Entry(key, value), null);
            size++;
            return;
        }
        Node node = subtreeSearch(root, key); // Encontrar o local para inserção
        if (key.equals(node.entry.key)) { // Atualizar valor se a chave já existe
            node.entry.value = value; // substitui
        } else {
            Node newNode = new Node(new Entry(key, value), node);
            if (key.compareTo(node.entry.key) < 0) node.left = newNode; // Inserir como filho esquerdo
            else node.right = newNode;  // Inserir como filho direito
            size++;
        }
    }
    // Método para remover uma chave da árvore
    public void remove(K key) {
        Node node = subtreeSearch(root, key);
        if (node == null || !node.entry.key.equals(key)) throw new IllegalArgumentException("Chave não encontrada");

    

        if (node.left != null && node.right != null) {  
            Node predecessor = subtreeMax(node.left);
            node.entry = predecessor.entry;
            node = predecessor;
        }

        Node child = (node.left != null) ? node.left : node.right;
        if (child != null) child.parent = node.parent;

        if (node == root) root = child;
        else if (node == node.parent.left) node.parent.left = child;
        else node.parent.right = child;

        size--;
    }
    // Método para encontrar um nó com uma determinada chave na árvore
    private Node subtreeSearch(Node node, K key) {
        if (node == null) return null;
        if (key.equals(node.entry.key)) return node;
        if (key.compareTo(node.entry.key) < 0) {
            return (node.left != null) ? subtreeSearch(node.left, key) : node;
        } else {
            return (node.right != null) ? subtreeSearch(node.right, key) : node;
        }
    }

    private Node subtreeMin(Node node) {
        while (node.left != null) node = node.left;
        return node;
    }

    private Node subtreeMax(Node node) {
        while (node.right != null) node = node.right;
        return node;
    }

    public Entry findMin() {
        if (root == null) return null;
        return subtreeMin(root).entry;
    }

    public int size() {
        return size;
    }

    public void inOrderTraversal() {
        inOrderTraversal(root);
    }

    private void inOrderTraversal(Node node) {
        if (node != null) {
            inOrderTraversal(node.left);
            System.out.println(node.entry.key + " => " + node.entry.value);
            inOrderTraversal(node.right);
        }
    }
}

```

#### Testando a Arvore de Busca

```java
public class Main {
  public static void main(String[] args) {
      // Cria uma instância da árvore binária de pesquisa com Integer (chaves) e String (valores)
      TreeMap<Integer, String> map = new TreeMap<>();

      // Insere elementos na árvore:
      map.put(50, "maçã");  // Inserir a chave 50 com valor "maçã"
      map.put(30, "banana"); // Inserir a chave 30 com valor "banana"
      map.put(70, "laranja"); // Inserir a chave 70 com valor "laranja"
      map.put(20, "kiwi");   // Inserir a chave 20 com valor "kiwi"
      map.put(40, "uva");    // Inserir a chave 40 com valor "uva"

      // Obtém o valor associado à chave 30:
      System.out.println("Valor da chave 30: " + map.get(30));

      // Encontra e imprime o Entry com a menor chave (mínimo):
      System.out.println("Valor mínimo: " + map.findMin());

      // Remove a chave 30:
      map.remove(30);

      // Imprime os elementos restantes em ordem:
      System.out.println("\nApós remover a chave 30:");
      map.inOrderTraversal();
  }
}
```